In [1]:
# get data lists
from nipype.interfaces.ants import ANTS
from nipype.interfaces import fsl
import nipype.interfaces.io as nio

#PD_LIST='/codes/PD_template_subjects.list';
current_group='PD'
SUB_LIST = '/codes/'+current_group+'_template_subjects.list'
with open(SUB_LIST, 'r') as f_sub:
    sub_list = f_sub.readlines()
sub_list = [x[0:-1] for x in sub_list]

# file import
ds_T1 = nio.DataGrabber(infields=['subject_id', 'subject_id'])
ds_T1.inputs.base_directory = '/data/'+current_group # database
ds_T1.inputs.template = '%s/anat/%s_desc-preproc_T1w.nii.gz' # from cwd
ds_T1.inputs.sort_filelist = True
ds_T1.inputs.subject_id = sub_list
res_T1 = ds_T1.run()
t1_list=res_T1.outputs.outfiles
# file import
ds_9c = nio.DataGrabber(infields=['subject_id', 'subject_id'])
ds_9c.inputs.base_directory = '/data/'+current_group # database
ds_9c.inputs.template = '%s/anat/%s_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz' # from cwd
ds_9c.inputs.sort_filelist = True
ds_9c.inputs.subject_id = sub_list
res_9c = ds_9c.run()
res_9c_list=res_9c.outputs.outfiles
# file import
ds_mask = nio.DataGrabber(infields=['subject_id', 'subject_id'])
ds_mask.inputs.base_directory = '/data/'+current_group # database
ds_mask.inputs.template = '%s/anat/%s_desc-brain_mask.nii.gz'# from cwd
ds_mask.inputs.sort_filelist = True
ds_mask.inputs.subject_id = sub_list
res_mask = ds_mask.run()
t1_mask_list=res_mask.outputs.outfiles

In [19]:
import h5py
res_9c_list[0]
warp_file='/data/PD/sub-0094/anat/sub-0094_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'

f = h5py.File(warp_file,'r')
print(f.keys())
d=f['TransformGroup']
print(d['0'].keys())
#a = f['data'][:]
#f.close()

<KeysViewHDF5 ['HDFVersion', 'ITKVersion', 'OSName', 'OSVersion', 'TransformGroup']>
<KeysViewHDF5 ['TransformType']>


In [43]:
# Jacobian of deformation field
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage
import os,time
t1_deformed_jacobian='/output/sMRI/test/Jacobian.nii.gz'
warp_file='/data/PD/sub-0094/anat/sub-0094_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'
jacobian = CreateJacobianDeterminantImage()
jacobian.inputs.imageDimension = 3
jacobian.inputs.deformationField = warp_file
jacobian.inputs.outputImage = t1_deformed_jacobian
jacobian.inputs.num_threads = 6
jacobian.cmdline
jacobian.run()


200818-23:04:35,359 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.359621:HDF5-DIAG: Error detected in HDF5 (1.8.7) thread 0:
200818-23:04:35,367 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.366780:  #000: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5D.c line 334 in H5Dopen2(): not found
200818-23:04:35,371 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.366780:    major: Dataset
200818-23:04:35,383 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.366780:    minor: Object not found
200818-23:04:35,384 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.366780:  #001: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gloc.c line 430 in H5G_loc_find(): can't find object
200818-23:04:35,386 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.366780:    major: Symbol table
200818-23:04:35,388 nipype.interface INFO:
	 stderr 2020-08-18T23:04:35.366780:    minor: Object not found
200818-23:04:35,389 nipype.interface INFO:
	 stderr 2020-08-1

RuntimeError: Command:
CreateJacobianDeterminantImage 3 /data/PD/sub-0094/anat/sub-0094_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5 /output/sMRI/test/Jacobian.nii.gz
Standard output:

Standard error:
HDF5-DIAG: Error detected in HDF5 (1.8.7) thread 0:
  #000: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5D.c line 334 in H5Dopen2(): not found
    major: Dataset
    minor: Object not found
  #001: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gloc.c line 430 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #002: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gtraverse.c line 905 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #003: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gtraverse.c line 799 in H5G_traverse_real(): component not found
    major: Symbol table
    minor: Object not found
terminate called after t

200818-23:03:46,914 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:HDF5-DIAG: Error detected in HDF5 (1.8.7) thread 0:
200818-23:03:46,918 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:  #000: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5D.c line 334 in H5Dopen2(): not found
200818-23:03:46,920 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:    major: Dataset
200818-23:03:46,926 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:    minor: Object not found
200818-23:03:46,931 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:  #001: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gloc.c line 430 in H5G_loc_find(): can't find object
200818-23:03:46,935 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:    major: Symbol table
200818-23:03:46,944 nipype.interface INFO:
	 stderr 2020-08-18T23:03:46.913895:    minor: Object not found
200818-23:03:46,946 nipype.interface INFO:
	 stderr 2020-08-1

RuntimeError: Command:
CreateJacobianDeterminantImage 3 /data/PD/sub-0094/anat/sub-0094_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5 /output/sMRI/test/Jacobian.nii.gz
Standard output:

Standard error:
HDF5-DIAG: Error detected in HDF5 (1.8.7) thread 0:
  #000: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5D.c line 334 in H5Dopen2(): not found
    major: Dataset
    minor: Object not found
  #001: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gloc.c line 430 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #002: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gtraverse.c line 905 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #003: /tmp/ANTs/build/ITKv4/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Gtraverse.c line 799 in H5G_traverse_real(): component not found
    major: Symbol table
    minor: Object not found
terminate called after t